In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import requests
import json
import dateutil.parser
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import tweepy
import datetime
import pandas as pd
import re

In [ ]:
#Function to get twitter data

def twitter_scrape(tags='$AAPL, Apple', from_date=None, until_date=None, today=False):
  '''This function scrapes twitter data which mentions the selected company.
  It gets the data for the past three days and returns a dataframe.
  INPUT: NONE
  OUTPUT: dataframe with text from twitter mentioning the selected company. One row per tweet. '''

  access_token = "1289900179701800960-2lujUQFC8RbuCcOgDMbloRzYPQY7ve"
  access_token_secret = "7hWUucfMm6oYutWWTBWdCu0RbIIJylfHOM6F0d8CtgH3e"
  consumer_key = "SJVSihso9VGRMpCC1acdbGk5M"
  consumer_secret = "TgQLnyhlEwQSIL9tsBVxwW7NNnxWhO5YaYhLN8brlH1WVjiw4x"


  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_token_secret)
  api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
  

  #Dates for scraping tweets: from three days ago until today
  startDate=datetime.datetime.today()-datetime.timedelta(days=7)
  endDate=datetime.datetime.today()
 
  #if today == True:
  #   start_date = ddatetime.datetime.today()
  #   end_date = datetime.datetime.today()

  #Scrape the data
  text  =[]
  for tweet in tweepy.Cursor(api.search, q=tags, lang="en").items(100):
      #keep only the past 3 days
      if tweet.created_at <= endDate and tweet.created_at >= startDate:
        text.append([tweet.created_at, tweet.text])


  dfv = pd.DataFrame(text, columns = ['date', 'text'])
  dfv.date = pd.to_datetime(dfv.date)
  dfv = dfv.set_index('date')

  #filter date
  #if today==True:
   # day = datetime.date.today()
   # mask = (df_full.index == day)
   # df_full  = df_full.loc[mask]
 
  #elif from_date != None:
   #  start_date = dateutil.parser.parse(from_date).date()
    # end_date = dateutil.parser.parse(until_date).date()
    # mask = (df_full.index >= start_date) & (df_full.index <= end_date)
     #df_full  = df_full.loc[mask]

  #Perform basica cleaning: remove @mentions, #hastags and URLs

  dfv.text = dfv.text.str.replace('@[A-Za-z0–9]+', '', regex=True)
  dfv.text = dfv.text.str.replace('#', '', regex=True)
  dfv.text = dfv.text.str.replace('RT[\s]+', '', regex=True)

  #Return in the form of a dataframe
  #text_df = pd.DataFrame({'text': text_series})
  
  return(dfv)

In [ ]:
#function to get news headliens

def scrape_news(from_date=None, until_date=None, today=False): 
  #  To fetch the top headlines
  #top_headlines_url = 'https://newsapi.org/v2/top-headlines'
  # To fetch news articles
  everything_news_url = 'https://newsapi.org/v2/everything'
  # To retrieve the sources
  #sources_url = 'https://newsapi.org/v2/sources'

  # The headers remain the same for all the requests
  headers = {'Authorization': '48f69d1519234b22bd0414da9c03d379'}
  headlines_payload = {'q': ['apple', 'Apple', '$AAPL'], 'sources': ['bloomberg', 'cnn', 'the-wall-street-journal', 'CNBC', 'Google News', 'MarketWatch', 'Seeking Alpha', 'business-insider'], 'sortBy': 'date'} #'category': 'business', 
  #sources_payload = {'category': 'general', 'language': 'en', 'country': 'us'}
  response = requests.get(url=everything_news_url, headers=headers, params=headlines_payload)
  # Convert response to a pure json string
  response_json_string = json.dumps(response.json())
    # A json object is equivalent to a dictionary in Python
    # retrieve json objects to a python dict
  response_dict = json.loads(response_json_string)
  #print(response_dict)
 # Info about articles is represented as an array in the json response
  # A json array is equivalent to a list in python
  # We want info only about articles
  news_list = response_dict['articles']
  df = pd.DataFrame(news_list)
  dfnew = df[['publishedAt', 'title', 'description']]
  dfnew['date'] = pd.to_datetime(dfnew['publishedAt'])
  dfnew['date'] = dfnew.date.dt.date
  df = dfnew[['date', 'title']]
  df = df.rename(columns={'title': 'text'})
  df = df.set_index('date')

  #FINVIZ
  finwiz_url = 'https://finviz.com/quote.ashx?t=AAPL'

  news_tables = {}
  req = Request(url=finwiz_url,headers={'user-agent': 'my-app/0.0.1'})
  response = urlopen(req)
  # Read the contents of the file into 'html'
  html = BeautifulSoup(response, features='lxml')
  # Find 'news-table' in the Soup and load it into 'news_table'
  news_table = html.find(id='news-table')
  parsed_news = []

  # Iterate through the news
  for x in news_table.findAll('tr'):
        # read the text from each tr tag into text
        # get text from a only
        text = x.a.get_text()
        # splite text in the td tag into a list
        date_scrape = x.td.text.split()
        # if the length of 'date_scrape' is 1, load 'time' as the only element

        if len(date_scrape) == 1:
            time = date_scrape[0]

        # else load 'date' as the 1st element and 'time' as the second
        else:
            date = date_scrape[0]
            time = date_scrape[1]
        
        parsed_news.append([date, text])
        
  dfv = pd.DataFrame(parsed_news, columns = ['date', 'text'])
  dfv.date = pd.to_datetime(dfv.date)
  dfv = dfv.set_index('date')

  df_full = pd.concat([df, dfv])

  #filter date
  if today==True:
    day = datetime.date.today()
    mask = (df_full.index == day)
    df_full  = df_full.loc[mask]
 
  elif from_date != None:
     start_date = dateutil.parser.parse(from_date).date()
     end_date = dateutil.parser.parse(until_date).date()
     mask = (df_full.index >= start_date) & (df_full.index <= end_date)
     df_full  = df_full.loc[mask]

  df_full.text = df_full.text.str.replace('[#@]', '', regex=True)
  df_full.text = df_full.text.str.replace('RT[\s]+', '', regex=True)

  return df_full 

In [ ]:
#function to get all data 

def getdata(twitter_scrape=twitterscrape, scrape_news=scrape_news):
  df1 = twitter_scrape()
  df2 = scrape_news()
  df_full = pd.concat([df1, df2])
  return df_full

In [ ]:
# TEXT CLEANING
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
import re

def preprocess(text, stem=False):
  '''This function does all the necessary preprocessing and cleaning of the texts
  so that they can be fed to the nlp for sentiment analysis.
  INPUT: a text (string)
  OUTPUT:  the cleaned text (string)'''

  # Define the stopwords and stemmer
  stop_words = stopwords.words("english")
  stemmer = SnowballStemmer("english")

  # Remove link,user and special characters
  text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    
  tokens = []
  for token in text.split():
    if token not in stop_words:
        if stem:
            tokens.append(stemmer.stem(token))
        else:
            tokens.append(token)
  return " ".join(tokens)

In [ ]:
# IMPORT
KERAS_MODEL = "model.h5"
WORD2VEC_MODEL = "model.w2v"
TOKENIZER_MODEL = "tokenizer.pkl"
ENCODER_MODEL = "encoder.pkl" 

# SENTIMENT
POSITIVE = "POSITIVE"
NEGATIVE = "NEGATIVE"
NEUTRAL = "NEUTRAL"
SENTIMENT_THRESHOLDS = (0.4, 0.7)

# KERAS
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
SEQUENCE_LENGTH = 300

#UTILITY
import time
import pickle

# nltk
import nltk

# Downloading the necessary for preprocessing
nltk.download('stopwords')

# UPDATE: this is the path were the models are stored
path = "/content/drive/Shared drives/NLP sentiment for stocks/model/"

def nlp():
  '''This function calls the scraper function which scrapes the past three days of tweets containing the selected company name.
  It performs a sentiment analysis of each tweet and gives the average over those three days.
  INPUT: NONE
  OUTPUT: sentiment average over the past three days.'''

  # Loads the saved keras model
  def load_models():

    # Load the tokenizer.
    file = open(path + TOKENIZER_MODEL, 'rb')
    tokenizer = pickle.load(file)
    file.close()
    # Load the model
    model = load_model(path + KERAS_MODEL)
    # Check its architecture
    model.summary()

    return tokenizer, model

  '''
  # Gives a label to the sentiment value
  def decode_sentiment(score, include_neutral=True):

    if include_neutral:        
        label = NEUTRAL
        if score <= SENTIMENT_THRESHOLDS[0]:
            label = NEGATIVE
        elif score >= SENTIMENT_THRESHOLDS[1]:
            label = POSITIVE

        return label
    else:
        return NEGATIVE if score < 0.5 else POSITIVE
  '''

  # Predicts the sentiment value for a tweet
  def predict(text, include_neutral=True):

    start_at = time.time()
    # Tokenize text
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=SEQUENCE_LENGTH)
    # Predict
    score = model.predict([x_test])[0]
    # Decode sentiment
    #label = decode_sentiment(score, include_neutral=include_neutral)

    return float(score)

  if __name__=="__main__":
    # Loading the models.
    tokenizer, model = load_models()

    # Getting the texts
    all_text = scrape()

    # Preprocessing data
    all_text.text = all_text.text.apply(lambda x: preprocess(x))

    # Calculating the sum of scores from all tweets over the past three days
    score = 0
    for text in all_text.text:
      score = score + predict(text)

    # Calculating the average score for the sentiment over the past three days
    score = score/len(all_text)

    return(score)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
print(nlp())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 300)          87125700  
_________________________________________________________________
dropout (Dropout)            (None, 300, 300)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 87,286,201
Trainable params: 160,501
Non-trainable params: 87,125,700
_________________________________________________________________
1
1
0.874923825263977
